In [13]:
import pandas as pd
import pandas_ta as ta
import random



candlestick_frame = 12
pnl_threshold = 3


df = pd.read_csv('../../data/ETHUSDT-15m.csv')

try:
    df.ta.strategy("all")
except Exception as e:
    print(e)

    ts = list(df['time'])
    open = list(df['open'])
    high = list(df['high'])
    low = list(df['low'])
    close = list(df['close'])
    volume = list(df['volume'])
    tot = len(ts)
    long_runup_lst = []
    long_drawdown_lst = []
    short_runup_lst = []
    short_drawdown_lst = []

    for idx in range(tot):
        if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
            max_high = max(high[idx:idx+candlestick_frame])
            min_low = min(low[idx:idx+candlestick_frame])
            entry_price = open[idx]
            long_runup_lst.append(round((max_high*100/entry_price)-100, 6))
            long_drawdown_lst.append(round((min_low*100/entry_price)-100, 6))
            short_runup_lst.append(round((entry_price*100/min_low)-100, 6))
            short_drawdown_lst.append(round((entry_price*100/max_high)-100, 6))
        else:
            long_runup_lst.append(0)
            long_drawdown_lst.append(0)
            short_runup_lst.append(0)
            short_drawdown_lst.append(0)     


    long=[]
    short=[]
    dont_trade=[]
    signal = []

    for idx in range(tot):
        if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
            if long_runup_lst[idx] >= pnl_threshold:
                signal.append('long')
            elif short_runup_lst[idx] >= pnl_threshold:
                signal.append('short')
            else:
                signal.append('dont_trade')
        else:
            signal.append('dont_trade')


    df['signal'] = signal

    long_indices = df[df['signal'].str.contains('long', case=False)].index
    short_indices = df[df['signal'].str.contains('short', case=False)].index
    dont_trade_indices = list(df[df['signal'].str.contains('dont_trade', case=False)].index)
    num_indices_to_pick  = len(dont_trade_indices) - min([len(long_indices), len(short_indices)])
    random_indices = random.sample(dont_trade_indices, num_indices_to_pick)
    df = df.drop(random_indices)

    df.reset_index(drop=True, inplace=True)

    # unnecessary_features = ['CDL_2CROWS','TRIMA_10','TOS_STDEVALL_U_3','PCTRET_1','CDL_RICKSHAWMAN','CDL_ONNECK','CDL_MORNINGSTAR','KAMA_10_2_30','SQZ_OFF','EMA_10','CDL_3BLACKCROWS','SQZ_NO','INC_1','TOS_STDEVALL_U_1','TOS_STDEVALL_U_2','TOS_STDEVALL_L_3','CDL_ABANDONEDBABY','CDL_GRAVESTONEDOJI', 'CDL_INVERTEDHAMMER', 'OHLC4', 'CDL_THRUSTING', 'CDL_STALLEDPATTERN', 'CDL_LONGLEGGEDDOJI', 'CDL_CLOSINGMARUBOZU', 'CDL_BREAKAWAY', 'CDL_CONCEALBABYSWALL', 'CDL_COUNTERATTACK', 'CDL_DARKCLOUDCOVER', 'CDL_TRISTAR', 'CDL_RISEFALL3METHODS', 'SSF_10_2', 'LR_14', 'DEC_1', 'FWMA_10', 'AOBV_SR_2', 'HWMA_0.2_0.1_0.1', 'HL2', 'HLC3', 'CDL_DOJISTAR', 'SQZPRO_NO', 'CDL_IDENTICAL3CROWS', 'CDL_TAKURI','CDL_KICKINGBYLENGTH','CDL_KICKING','CDL_UPSIDEGAP2CROWS', 'CDL_SPINNINGTOP','CDL_HARAMICROSS','CDL_PIERCING','CDL_MORNINGDOJISTAR','CDL_HARAMI','CDL_SEPARATINGLINES','CDL_SHOOTINGSTAR','SUPERTd_7_3.0','CDL_STICKSANDWICH','CDL_ADVANCEBLOCK','THERMOs_20_2_0.5','CDL_EVENINGDOJISTAR','MIDPOINT_2','BBM_5_2.0','AOBV_LR_2','AMATe_SR_8_21_2','ITS_9','SQZPRO_OFF','CDL_EVENINGSTAR','CDL_GAPSIDESIDEWHITE','SINWMA_14']
    features = ['signal','DPO_20','LOGRET_1','NATR_14','SLOPE_1','BBB_5_2.0','ICS_26','BBP_5_2.0','PGO_14','BEARP_13','J_9_3','EFI_13','BOP','DMN_14','BULLP_13','DMP_14','CG_10','TRIXs_30_9','SMIo_5_20_5','TRUERANGE_1','EBSW_40_10','PVOL','THERMO_20_2_0.5','CHOP_14_1_100','PVI_1','VHF_28','ADX_14','TRIX_30_9','PVOh_12_26_9','PPOh_12_26_9','UO_7_14_28','INERTIA_20_14','SKEW_30','KVOs_34_55_13', 'Ul_14','HILOs_13_21','CMF_20','KURT_30','RVI_14','HILOI_13_21','PVO_12_26_9','PVOs_12_26_9','MASSI_9_25','TSIs_13_25_13','wo_34_55_13','ISB_26','THERMOma_20_2_0.5','SUPERTs_7_3.0','NVI_1','BR_26','MFI_14','STDEV_30','ADOSC_3_10','OBVe_12','AR_26','SUPERTI_7_3.0','TOS_STDEVALL_LR','PDIST','OBV','STCstoch_10_12_26_0.5','STOCHRSld_14_14_3_3','ROC_10','STC_10_12_26_0.5','PSARI_0.02_0.2','PPO_12_26_9','CCI_14_0.015','MACDh_12_26_9','CMO_14','ER_10']
    ndf = pd.DataFrame()
    for i in features:
        ndf[i] = list(df[i])
    df = df.drop(columns = ['time', 'open', 'high', 'low', 'close', 'volume'], axis=1)
    
    print('saving dataset.csv....')
    dataset_fn = f'../../data/dataset.csv'
    pd.DataFrame(ndf).to_csv(dataset_fn, index=False)




0it [00:00, ?it/s]

[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.


/usr/lib/python3.10/multiprocessing/pool.py:48: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return list(map(*args))
123it [00:19,  6.39it/s]


'RangeIndex' object has no attribute 'to_period'
saving dataset.csv....


In [14]:

df = pd.read_csv('../../data/ETHUSDT-15m.csv')
features = ['signal','DPO_20','LOGRET_1','NATR_14','SLOPE_1','BBB_5_2.0','ICS_26','BBP_5_2.0','PGO_14','BEARP_13','J_9_3','EFI_13','BOP','DMN_14','BULLP_13','DMP_14','CG_10','TRIXs_30_9','SMIo_5_20_5','TRUERANGE_1','EBSW_40_10','PVOL','THERMO_20_2_0.5','CHOP_14_1_100','PVI_1','VHF_28','ADX_14','TRIX_30_9','PVOh_12_26_9','PPOh_12_26_9','UO_7_14_28','INERTIA_20_14','SKEW_30','KVOs_34_55_13', 'Ul_14','HILOs_13_21','CMF_20','KURT_30','RVI_14','HILOI_13_21','PVO_12_26_9','PVOs_12_26_9','MASSI_9_25','TSIs_13_25_13','wo_34_55_13','ISB_26','THERMOma_20_2_0.5','SUPERTs_7_3.0','NVI_1','BR_26','MFI_14','STDEV_30','ADOSC_3_10','OBVe_12','AR_26','SUPERTI_7_3.0','TOS_STDEVALL_LR','PDIST','OBV','STCstoch_10_12_26_0.5','STOCHRSld_14_14_3_3','ROC_10','STC_10_12_26_0.5','PSARI_0.02_0.2','PPO_12_26_9','CCI_14_0.015','MACDh_12_26_9','CMO_14','ER_10']
c=list(df.columns)
for i in features:
    if i not in c:
        print(i)

signal
DPO_20
LOGRET_1
NATR_14
SLOPE_1
BBB_5_2.0
ICS_26
BBP_5_2.0
PGO_14
BEARP_13
J_9_3
EFI_13
BOP
DMN_14
BULLP_13
DMP_14
CG_10
TRIXs_30_9
SMIo_5_20_5
TRUERANGE_1
EBSW_40_10
PVOL
THERMO_20_2_0.5
CHOP_14_1_100
PVI_1
VHF_28
ADX_14
TRIX_30_9
PVOh_12_26_9
PPOh_12_26_9
UO_7_14_28
INERTIA_20_14
SKEW_30
KVOs_34_55_13
Ul_14
HILOs_13_21
CMF_20
KURT_30
RVI_14
HILOI_13_21
PVO_12_26_9
PVOs_12_26_9
MASSI_9_25
TSIs_13_25_13
wo_34_55_13
ISB_26
THERMOma_20_2_0.5
SUPERTs_7_3.0
NVI_1
BR_26
MFI_14
STDEV_30
ADOSC_3_10
OBVe_12
AR_26
SUPERTI_7_3.0
TOS_STDEVALL_LR
PDIST
OBV
STCstoch_10_12_26_0.5
STOCHRSld_14_14_3_3
ROC_10
STC_10_12_26_0.5
PSARI_0.02_0.2
PPO_12_26_9
CCI_14_0.015
MACDh_12_26_9
CMO_14
ER_10
